In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train = pd.read_csv('../input/jobathon-june-22/train_wn75k28.csv')
train

In [3]:
train.isna().sum()

In [4]:
train.corr()

In [5]:
train.hist('products_purchased')

In [6]:
train.dtypes

In [7]:
train['created_at'] = pd.to_datetime(train['created_at'])
train['signup_date'] = pd.to_datetime(train['signup_date'], errors='coerce')
train.dtypes

In [8]:
train.isna().sum()

In [9]:
train.hist('created_at')

In [10]:
train.hist('signup_date')

In [11]:
train.hist('buy')

In [12]:
train['buy'].value_counts(normalize=True)

In [13]:
first_column = train.pop('buy')
train.insert(0, 'buy', first_column)
train.drop(['id'],axis=1, inplace = True)

In [14]:
train.head()

In [16]:
import datetime as dt
train['day_created'] = train['created_at'].dt.day
train['month_created'] = train['created_at'].dt.month
train['year_created'] = train['created_at'].dt.year
train['weekday_created'] = train['created_at'].dt.weekday
train['day_of_year_created'] = train['created_at'].dt.dayofyear

In [17]:
train

In [18]:
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
train['day_of_year_created'].plot.hist()

In [20]:
train['month_created'].plot.hist()

In [21]:
train['day_created'].plot.hist()

In [22]:
train['weekday_created'].plot.hist(edgecolor='k').autoscale(enable=True, axis='both', tight=True)

In [23]:
train['day_signup'] = train['signup_date'].dt.day
train['month_signup'] = train['signup_date'].dt.month
train['year_signup'] = train['signup_date'].dt.year
train['weekday_signup'] = train['signup_date'].dt.weekday
train['day_of_year_signup'] = train['signup_date'].dt.dayofyear

In [24]:
train.head()

In [25]:
train.drop(['created_at','signup_date'],axis=1, inplace=True)

In [26]:
X = train.drop("buy", axis=1)
y = train["buy"]

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

In [29]:
from catboost import CatBoostClassifier

model_cb = CatBoostClassifier(task_type='CPU', iterations=200, 
                              random_state = 2021, 
                              eval_metric="F1")

In [30]:
model_cb.fit(X_train, y_train,  plot=True, 
             eval_set=(X_val, y_val))

In [31]:
y_pred = model_cb.predict(X_val)

In [32]:
from sklearn.metrics import f1_score, accuracy_score
f1_score(y_val, y_pred)

In [33]:
test = pd.read_csv('../input/jobathon-june-22/test_Wf7sxXF.csv')
test.head()

In [34]:
test['created_at'] = pd.to_datetime(test['created_at'])
test['signup_date'] = pd.to_datetime(test['signup_date'], errors='coerce')

In [36]:
test.drop('id',axis=1,inplace=True)

In [37]:
test['day_created'] = test['created_at'].dt.day
test['month_created'] = test['created_at'].dt.month
test['year_created'] = test['created_at'].dt.year
test['weekday_created'] = test['created_at'].dt.weekday
test['day_of_year_created'] = test['created_at'].dt.dayofyear
test['day_signup'] = test['signup_date'].dt.day
test['month_signup'] = test['signup_date'].dt.month
test['year_signup'] = test['signup_date'].dt.year
test['weekday_signup'] = test['signup_date'].dt.weekday
test['day_of_year_signup'] = test['signup_date'].dt.dayofyear

In [39]:
test.drop(['created_at','signup_date'],axis=1,inplace=True)

In [40]:
y_pred = model_cb.predict(test)

In [74]:
test = pd.read_csv('../input/jobathon-june-22/test_Wf7sxXF.csv')

In [78]:
sub = pd.DataFrame()

In [79]:
sub['id'] = test['id']

In [81]:
sub['buy'] = y_pred

In [84]:
sub.to_csv('submit01.csv', index=False)